# 02_supervised_models.ipynb — Duplicate classification

Цей ноутбук: supervised ML для задачі duplicate / not duplicate. Baseline + 2+ моделі, підбір параметрів (за потреби), метрики, таблиця експериментів.

## Installs (Colab)

In [ ]:
!pip install -q datasets scikit-learn pandas matplotlib lightgbm xgboost

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


## Load dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "sentence-transformers/stackexchange-duplicates",
    "title-title-pair"
)

# Convert to pandas for convenience
df = dataset["train"].to_pandas()
df.head()

###Робимо класифікацію (duplicate / not duplicate)

Зараз датасет містить тільки пари-дублікатів (позитивні приклади)(huggingface.co).

Щоб зробити класифікацію (duplicate / not duplicate), ми:

1. Позначаємо всі існуючі пари як label = 1.

2. Генеруємо negative pairs:

* для кожного title1 випадково беремо title2 з іншого рядка

* такі пари з великою ймовірністю не є дублікатами - label = 0.

3. Об’єднуємо в один датафрейм.

In [ ]:
# Позитиви
pos = df[["title1", "title2"]].copy()
pos["label"] = 1

# Негативи: перемішуємо title2
neg = df[["title1", "title2"]].copy()
neg = neg.sample(frac=1.0, random_state=42).reset_index(drop=True)
neg["label"] = 0

Щоб не було ідеального співпадіння з позитивами, можна ще раз перемішати/зміксувати.

In [ ]:
full_df = pd.concat([pos, neg], axis=0).sample(frac=1.0, random_state=42).reset_index(drop=True)

full_df.head(), full_df["label"].value_counts()

Ми отримали великий, збалансований, якісний supervised датасет для задачі визначення дублікатів запитань.

**label**

`0 - 304525`

`1 - 304525`

###Підготовка тексту: об’єднуємо заголовки

Для простого бейзлайна зручно зліпити обидва заголовки в один рядок - модель отримає контекст "пара запитань".

In [ ]:
df = full_df.copy()

# Об'єднуємо заголовки в єдиний текстовий вхід
df["text"] = df["title1"] + " [SEP] " + df["title2"]

df[["title1", "title2", "text", "label"]].head()

##Train / Validation / Test split

Зробимо 70% train, 15% val, 15% test.

In [ ]:
X = df["text"].values
y = df["label"].values

In [ ]:
# спочатку train+temp / test
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

In [ ]:
# тепер train / val
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1765,  # 0.1765 * 0.85 ≈ 0.15
    random_state=42, stratify=y_train_val
)

In [ ]:
print("Train: ", len(X_train))
print("Validation: ", len(X_val))
print("Test: ", len(X_test))

##Models

###TF-IDF + Logistic Regression

####TF-IDF векторизація

Бейслайн: unigrams + bigrams, обмежимо розмір словника, щоб не "вбити" пам’ять у Colab.

In [ ]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=100_000,
    min_df=5,             # ігноруємо рідкісні токени
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf   = tfidf.transform(X_val)
X_test_tfidf  = tfidf.transform(X_test)

print(f"Train_tfidf: { X_train_tfidf.shape }")
print(f"Validation_tfidf: { X_val_tfidf.shape }")
print(f"Test_tfidf: { X_test_tfidf.shape }")

**У результаті ми отримали розріджені матриці ознак таких розмірів:**

Train: (426 319 × 100 000)

Validation: (91 373 × 100 000)

Test: (91 358 × 100 000)

Це означає, що кожен текстовий приклад представлено у просторі з 100 000 TF-IDF ознак.
Попри високу розмірність, Logistic Regression добре масштабується для таких sparse-матриць і може служити сильною базовою моделлю.

####Logistic Regression

In [ ]:
logreg = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    verbose=0,
    class_weight="balanced",
    C=3.0
)

logreg.fit(X_train_tfidf, y_train)

**Оцінка на validation set**

Використаємо accuracy, precision, recall, F1, плюс confusion matrix.

In [ ]:
# Прогнози
y_val_pred = logreg.predict(X_val_tfidf)

# Класичний звіт
print(classification_report(y_val, y_val_pred, digits=4))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_val, y_val_pred)

plt.figure(figsize=(4,4))
sns.heatmap(cm, annot=True, fmt="d", cbar=False,
            xticklabels=["not duplicate (0)", "duplicate (1)"],
            yticklabels=["not duplicate (0)", "duplicate (1)"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix (Validation)")
plt.show()

In [ ]:
acc = accuracy_score(y_val, y_val_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_val, y_val_pred, average="binary")

print(f"Validation accuracy: {acc:.4f}")
print(f"Validation precision: {prec:.4f}")
print(f"Validation recall: {rec:.4f}")
print(f"Validation F1: {f1:.4f}")

**Спостереження**

Попри використання `class_weight="balanced"` + `С=3.0`, baseline модель TF-IDF + LogisticRegression повністю провалилася на задачі визначення дублікатів запитань.

Модель передбачила лише клас 0 ("не дублікат") для всіх прикладів. Це призвело до:

`precision = 0`, `recall = 0` для класу 1

`accuracy ≈ 0.5` (випадкове вгадування при збалансованих класах)

Причина: TF-IDF моделює лише лексичну схожість, а більшість дублікатів у нашому датасеті мають різні формулювання, але подібний зміст.
Логістична регресія не здатна уловити семантичні зв’язки і працює на рівні випадковості.

###XGBoost / LightGBM on engineered text features

In [ ]:
df_engineered = df.copy()

# Токенізація довжин
df_engineered["len1"] = df_engineered["title1"].str.split().apply(len)
df_engineered["len2"] = df_engineered["title2"].str.split().apply(len)
df_engineered["len_diff"] = np.abs(df_engineered["len1"] - df_engineered["len2"])

In [ ]:
# TF-IDF для cosine similarity
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=100_000,
    min_df=5
)

In [ ]:
tfidf.fit(pd.concat([df_engineered["title1"], df_engineered["title2"]]))

X1_tfidf = tfidf.transform(df_engineered["title1"])
X2_tfidf = tfidf.transform(df_engineered["title2"])

In [ ]:
# L2-нормалізуємо обидві матриці построчно
from sklearn.preprocessing import normalize
X1_norm = normalize(X1_tfidf, norm="l2", axis=1)
X2_norm = normalize(X2_tfidf, norm="l2", axis=1)

In [ ]:
# Елементний добуток і сума по фічах -> косинус схожості для кожної пари
cos_sim = X1_norm.multiply(X2_norm).sum(axis=1)

In [ ]:
# Перетворюємо у вектор форми (n_samples, 1)
cos_sim = np.array(cos_sim).ravel().reshape(-1, 1)

cos_sim.shape

**Примітка**

Під час першої спроби обчислити cosine similarity між TF-IDF-векторами ми використали sklearn.metrics.pairwise.cosine_similarity(X1, X2).
Однак ця функція рахує попарну схожість між усіма рядками двох матриць, тобто намагається побудувати матрицю розміром N × N (у нашому випадку ≈ 600k × 600k), що повністю вичерпує оперативну пам’ять у Google Colab.

Щоб обійти цю проблему, ми нормалізували TF-IDF-вектори построчно та обчислили скалярний добуток відповідних рядків:

* нормалізація X1 і X2 до L2-норми,

* множення X1_norm.multiply(X2_norm)

* сума по фічах для кожного рядка.

Такий підхід дозволяє обчислити cosine similarity рядок-до-відповідного-рядка в sparse-форматі, не виходячи за межі пам’яті.

In [ ]:
#Збираємо фінальну матрицю ознак
X_feats = np.hstack([
    cos_sim,  # (609050, 1)
    df_engineered["len1"].values.reshape(-1, 1),
    df_engineered["len2"].values.reshape(-1, 1),
    df_engineered["len_diff"].values.reshape(-1, 1)
])

y = df_engineered["label"].values

X_feats.shape

In [ ]:
#Робимо train / val / test split
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_feats, y, test_size=0.15, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1765,
    random_state=42, stratify=y_train_val
)

X_train.shape, X_val.shape, X_test.shape

Далі запускаємо XGBoost на цих фічах

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(
    n_estimators=400,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=1.0,
    eval_metric="logloss",
    n_jobs=-1
)

xgb.fit(X_train, y_train)

y_val_pred = xgb.predict(X_val)

print(classification_report(y_val, y_val_pred, digits=4))


LightGBM на тих самих фічах

In [ ]:
import lightgbm as lgb
lgb_model = lgb.LGBMClassifier(
    n_estimators=500,
    max_depth=-1,
    learning_rate=0.05,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=1.0,
    class_weight="balanced"
)

lgb_model.fit(X_train, y_train)

y_val_pred_lgb = lgb_model.predict(X_val)
print(classification_report(y_val, y_val_pred_lgb, digits=4))

**Спостереження**

Обидві моделі показали значно кращі результати, ніж TF-IDF baseline.
Наприклад, XGBoost досяг:

* F1 для класу 0: 0.45

* F1 для класу 1: 0.45

* accuracy: 0.45

LightGBM показав подібні значення (~0.42).

Хоча ці результати все ще далекі від бажаних, вони демонструють, що навіть прості семантичні та структурні ознаки дозволяють класичним моделям робити осмислені прогнози.
Втім, для задачі визначення дублікатів запитань класичні ML-підходи суттєво обмежені та не здатні уловити глибоку внутрішню семантику тексту.

In [ ]:
import os
os.makedirs("services/classifier/artifacts", exist_ok=True)

In [ ]:
import joblib
joblib.dump(tfidf, "services/classifier/artifacts/tfidf_vectorizer.joblib")
joblib.dump(lgb_model, "services/classifier/artifacts/classifier.joblib")